## 1. Building Agents
There are four types of agents we decided to implement. [aggiungere testo]

### 1.1 Agent Saver
[aggiungere testo]

In [ ]:
class Agent_Saver(mesa.Agent):
    def __init__(self, unique_id, model,
                 wealth, position, order,
                 wait_invest, invested):
        super().__init__(unique_id, model)
        
        # DOF Agent
        self.wealth = round(wealth * ( 1. + 0.1 * random.uniform(-1, 1) ), 2) # cash
        self.position = position # initial position, buy or sell
        self.order = order # quantities ordered
        
        # DOF Agent Saver
        self.wait_invest = wait_invest # timesteps of waiting before buying
        self.waiting = random.randint(1, 30) # not all investors invest at the sime timestep
        self.invested = invested
        
        #printing init stats
        print(f'{self.unique_id}: \t $ {round(self.wealth, 2)}')
        
    # Saver Functions
    # the savers keep the position since the beginning of simulation
    # and keep buying or selling during the simulation
    def invest(self):
        if self.position == 'buy':
            self.order = 5 # choose a quantity to buy
        elif self.position == 'sell':
            self.order = 5 # choose a quantity to sell
        
        # update remaining wealth
        self.wealth = self.wealth - self.order * model.Price
    
    # Step Function for saver agent
    # if they have no more wealth they're out of money to invest
    # assume savers have a constant outside income they invest periodically
    def step(self):
        if self.wealth <= 0:
            self.position = 'null'
        
        if self.invested == True:
            self.order = 0 # do not buy anymore but wait
            self.waiting -= 1
            if self.waiting == 0:
                self.invested = False
        else:
            self.wealth += 100 # income of a saver from a portion of its salary
            self.invest()
            self.invested = True
            self.waiting = self.wait_invest # reset counter for next investing

### 1.2 Agent Random Trader
[aggiungere testo]

In [ ]:
class Agent_Rnd_Trader(mesa.Agent):
    def __init__(self, unique_id, model,
                 wealth, position, order):
        super().__init__(unique_id, model)

        # DOF Agent
        self.wealth = wealth
        self.position = position # initial position, buy or sell
        self.order = order # quantities ordered
        
    # Random Trader Functions
    def rnd_trade(self):
        if random.uniform(0,1) < 0.5:
            self.position = 'buy'
            self.order = 1 #randomize also this later
        else:
            self.position = 'sell'
            self.order = 1
        
    # Step Function for random trader agent
    def step(self):
        self.rnd_trade()

### 1.3 Agent Follower
[aggiungere testo]

In [ ]:
class Agent_Follower(mesa.Agent):
    def __init__(self, unique_id, model,
                 wealth, position, order,
                 wallet, detrust, p_close, p_close_thr):
        super().__init__(unique_id, model)
        
        # DOF Agent
        self.wealth = round(wealth * ( 1. + 0.1 * random.uniform(-1, 1) ), 2)
        self.position = position
        self.order = order
        
        # DOF Agent Follower
        self.wallet = wallet
        self.detrust = detrust
        self.p_close = p_close
        self.p_close_thr = p_close_thr * random.uniform(0.5, 1.5)
        
        #printing init stats
        print(f'{self.unique_id}: \t $ {round(self.wealth, 2)}\t close_thr: {round(self.p_close_thr*100,1)}%')
        
    def follow(self):
        # ... choose among all agents which one to follow

        #random.seed(42) # reproduce randomicity
        
        list_agents = self.model.schedule.agents
        list_wealths = list()
        for i in range(len(list_agents)):
            list_wealths.append(list_agents[i].wealth)
        
        # create restricted list of most healthy agents
        top = 5
        top_list_idx = list()
        max_value = None        
        
        for j in range(top):
            max_value = max(list_wealths) # save max value
            max_idx = list_wealths.index(max_value) # save its index
            list_wealths[max_idx] = 0 # it's like removing it, but setting to zero maintains indices ordering
            
            top_list_idx.append(max_idx) # this list of indices makes it possible to obtain other infos on the hubs
        
        # choose an agent to follow who isn't himself
        idx_chosen = random.choice(top_list_idx)
        while list_agents[idx_chosen].unique_id == self.unique_id:
            idx_chosen = random.choice(top_list_idx)
        
        # the follower now trusts the agent he is following
        self.detrust = False        
        
        #print("Hi! I'm " + self.unique_id + " and i'm following " + list_agents[idx_chosen].unique_id)
        
        # copy the position of the chosen agent to follow
        self.position = list_agents[idx_chosen].position
        self.order = 1
        
        self.wealth = self.wealth - self.order * model.Price
        self.wallet = self.order * model.Price
        
    def check_profit(self):
        # if the follower is not satisfied with the gains he can unfollow, which means he chooses another to follow
        if self.position == 'buy':
            self.p_close = (model.Price - self.wallet)/self.wallet
        elif self.position == 'sell':
            self.p_close = (self.wallet - model.Price)/self.wallet
        
        if (model.Step_Gain < 0 and self.position == 'sell') or (model.Step_Gain > 0 and self.position == 'buy'):
            # keep position till p_close_thr is reached
            if self.p_close >= self.p_close_thr:
                self.close_position()
        else:
            #close position and set self.detrust = True
            self.close_position()
            self.detrust = True
            
    def close_position(self):
        if self.position == 'buy':
            self.position = 'sell'
            self.order = 1
        elif self.position == 'sell':
            self.position = 'buy'
            self.order = 1

        self.wealth = self.wealth + self.wallet * (1 + self.p_close)
        
    def step(self):
        self.wealth += 10
        
        if self.wealth <= 0:
            self.position = 'null'
        
        #if detrust is true then it means follower is looking for another agent to follow
        if self.detrust == True:
            self.follow()
        else:
            #check if profit was made
            self.check_profit()

### 1.4 Agent Whale
[aggiungere testo]

In [ ]:
class Agent_Whale(mesa.Agent):
    def __init__(self, unique_id, model,
                 wealth, position, order,
                 waiting, WLT, confidence,
                 wallet, p_close, p_close_thr):
        super().__init__(unique_id, model)

        #random.seed(42)

        # DOF Agent
        self.wealth = round(wealth * ( 1. + 0.1 * random.uniform(-1, 1) ), 2)
        self.position = position
        self.order = order
        
        # DOF Agent Whale
        self.wallet = wallet
        self.WLT = WLT
        self.waiting = waiting
        self.confidence = confidence # this is the confidence in the correctness of their own strategy
        self.p_close = p_close
        self.p_close_thr = p_close_thr * random.uniform(0.9, 1.1)
    
        # printing init stats
        print(f'{self.unique_id}: \t $ {round(self.wealth, 2)}\t close_thr: {round(self.p_close_thr*100,1)}%\t WLT: \t{self.WLT}')        
        
    def check_strat(self):
        if model.Step_Gain < 0:
            self.position = 'buy'
            self.order = 1
            if model.Net_Result < 0:
                self.waiting +=1
            else:
                self.waiting = 0
        else:
            self.position = 'sell'
            self.order = 1
            if model.Net_Result > 0:
                self.waiting +=1
            else:
                self.waiting = 0
        
        self.wealth = self.wealth - self.order * model.Price
        self.wallet = self.order * model.Price
        
        if self.waiting >= self.WLT:
            self.confidence = False
            self.waiting = 0
            
        if self.position == 'buy':
            self.p_close = (self.wealth - self.wallet)/self.wallet
        elif self.position == 'sell':
            self.p_close = (self.wallet - self.wealth)/self.wallet
            
        if self.p_close >= self.p_close_thr:
            self.change_strategy()
            self.confidence = True
    
    def change_strategy(self):
        if self.position == 'buy':
            self.position == 'sell'
            self.order = 1
        elif self.position == 'sell':
            self.position == 'buy'
            self.order = 1
            
        self.wealth = self.wealth - self.order * model.Price
        self.wallet = self.order * model.Price
        
    def close_position(self):
        if self.position == 'sell':
            self.position = 'buy'
            self.order = 1
        elif self.position == 'buy':
            self.position = 'sell'
            self.order = 1
            
        self.wealth = self.wealth + self.wallet * (1 + self.p_close)
        
    
    def step(self):
        self.wealth += 20
        
        if self.wealth <= 0:
            self.position = 'null'
          
        if self.confidence == True:
            self.check_strat()
        else:
            self.close_position()
            self.check_strat()